# EDA 과정





## 1.데이터 로딩

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/프로젝트1/vgames2.csv')

df.drop(['Unnamed: 0'],axis=1,inplace=True) # 보기 싫어서 삭제

df

## 2.데이터 분석

### 데이터 클리닝 과정

In [ ]:
print(df.isnull().sum()) # NAN값을 확인한다.

df.dropna(inplace= True) # NAN 데이터를 제거 했다. Year feature 데이터 나열이 알아보기 힘들어서 Year 데이터를 int로 바꾸기 위해 제거함
df.reset_index(drop=True,inplace=True) 

print(df['Year'].astype('int').unique()) # year 값에 다른게 끼어있는지 확인한다. / 값이 지수 곱으로 표시되어서 int로 바꿔확인한다.

print(df['NA_Sales'].unique(),df['EU_Sales'].unique(),df['JP_Sales'].unique(),df['Other_Sales'].unique()) # 각 sales 데이터 값에 다른 데이터가 끼어있는지 확인한다.

- 분석<br>
1) NAN 값이 전체 데이터에 비해선 매우 적다. NAN값을 가지고 있는 데이터 행을 전체 삭제하더라도 전체 데이터의 2% 정도의 손실 밖에 없기 때문에 삭제 해주는 편이 좋을 것이라고 판단된다.-> 사실상 분석을 위해 Year을 int로 바꾸는 과정에서 이미 NAN 데이터를 제거함<br><br>
2) Year 값에 네자리의 연도 외에도 2, 86, 8 같은 데이터들이 보인다.1\~21까지의 숫자는 2000년도 이후로 보이고 22\~99 사이의 숫자는 1900년도로 보는게 타당해 보인다.<br><br>
3) 전체 판매량의 단위는 M가 기준으로 보인다. 왜냐하면 중간 유니크 값인 480K의 단위를 1 기준으로 봤을 때 480,000이 나온다. 이는 다른 값들이 대부분 소수점 두자리라는 걸 생각해보면 지나치게 큰 값이고 K라고 가정했을 경우에도 480은 매우 큰 값이기 때문에 M 단위로 보고 480K를 0.48로 보는 것이 타당하다.


In [ ]:
# 2) 함수를 만들어 Year 값을 19xx,20xx 형식으로 바꿔주겠다.

y=df['Year'].astype('int') # Year feature를 추출해준다.

def Year2000(x):# 21 이하수를 20xx형식으로 바꿔주는 함수이다.
  if x <= 21:
    x= 2000+ x
  return x

def Year1900(x):# 22~99를 19xx형식으로 바꿔주는 함수이다.
  if (x > 21) & (x <=99):
    x= 1900 + x
  return x

y=y.apply(Year2000)
y=y.apply(Year1900)# 갹각 햠수 적용

print(y.unique()) # 제대로 바뀌었는지 체크

df['Year']=y # 바뀐 내용을 데이터 베이스에 넣어준다.

In [ ]:
# 3) sales feature의 형태를 M 단위 기준으로 치환해주겠다.
s = df[['NA_Sales','EU_Sales','JP_Sales','Other_Sales']] # 각 Sales feature 추출

def d_M(x):# M 삭제 함수
  x=str(x)
  return x.replace('M','')

def d_K(x): # K를 삭제하고 값을 1/1000 하는 함수
  x=str(x)
  return float((x.replace('K','e-03'))) # K 값을 e-03으로 바꿔주고 float로 바꿔주면 1/1000 적용된다.

s=s.applymap(d_M)
s=s.applymap(d_K)

df[['NA_Sales','EU_Sales','JP_Sales','Other_Sales']]=s # 수정 후 다시 df에 넣어준다.

# 데이터가 전부 numerical data로 이뤄졌는지 확인

df['NA_Sales'].astype('float')
df['EU_Sales'].astype('float')
df['JP_Sales'].astype('float')
df['Other_Sales'].astype('float')

# 문제없이 실행되어 data는 전부 numerical data이다.

### 이상치 확인 과정


- 중심값과 중앙값을 확인하자<br><br>
  1) Year<br>
  2) NA_Sales<br>
  3) EU_Sales<br>
  4) JP_Sales<br>
  5) Other_Sales

In [ ]:
print(df['Year'].describe()) # 평균값: 2006 중앙값: 2007 거의 차이가 없다 굳이 특이값을 자를 필요는 없을 것 같다.
print(df['NA_Sales'].describe())# 평균값: 0.265195 중앙값: 0.080000
print(df['EU_Sales'].describe())# 평균값: 0.147378 중앙값: 0.020000
print(df['JP_Sales'].describe())# 평균값: 0.078283 중앙값: 0.000000
print(df['Other_Sales'].describe())# 평균값: 0.048234 중앙값: 0.010000

# Year feature을 제외하면 나머지는 특이값을 조금 살펴볼 필요가 있다.

- boxplot을 그려 가시적으로 데이터 분포를 본다

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# 
plt.figure(figsize=(3,3))
sns.boxplot(data=df['Year'],color='red')

plt.figure(figsize=(3,3))
sns.boxplot(data=df['NA_Sales'],color='yellow')

plt.figure(figsize=(3,3))
sns.boxplot(data=df['EU_Sales'],color='green')

plt.figure(figsize=(3,3))
sns.boxplot(data=df['JP_Sales'],color='red')

plt.figure(figsize=(3,3))
sns.boxplot(data=df['Other_Sales'],color='red')

plt.show()

- 각 feature의 상위값을 뽑아 특이값을 확인하고 box그래프를 보며 위치를 대강 확인해본다.

In [ ]:
df.sort_values(by='NA_Sales',ascending=False).head(50)

In [ ]:
df.sort_values(by='EU_Sales',ascending=False).head(30)

In [ ]:
df.sort_values(by='JP_Sales',ascending=False).head(30)

In [ ]:
df.sort_values(by='Other_Sales',ascending=False).head(30)

In [ ]:
condition123= df['Publisher']=='Nintendo' # Nintendo 데이터의 개수를 세어본다

df[condition123]

- 결론: 이상치의 대부분은 Nintendo에 치우쳐져 있고 만일 이상치를 지운다면 Nintendo 데이터의 약 10% 이상을 잃을 것이다. 따라서 이상치를 제거하지 않고 진행하며 일단은 Nintendo의 시장 지배력이 상당히 크다고 판단하겠다.

# 지역에 따라서 선호하는 게임 장르

- 귀무가설인 '지역별로 각 장르는 차이가 없다'를 검정하여 기각 됐을 경우 누적합을 통해 가시적으로 선호하는 게임 장르를 확인한다.

### 1.통계 분석을 위해 지역별, 장르별 데이터를 나눠준다.

In [ ]:
genre = df['Genre'].unique()
df1= df[['Genre','NA_Sales','EU_Sales','JP_Sales','Other_Sales']]

In [ ]:

from scipy.stats import f_oneway
import matplotlib.pyplot as plt

list=[]
t= ['NA','EU','JP','Other']
for j,v in enumerate(t): # enumerate를 사용 j=0 , v= 'NA' 에서 시작한다------------------------------------------------------------------------------1번 for문
  a=df1.iloc[:,j+1] # NA_Sales 부터	t 열의 데이터를 차례대로 a에 입혀준다. df1의 0열이 Genre라서 1부터 시작한다.
  a=pd.DataFrame(a) # 위에서 받은 리스트를 데이터 프레임으로 만들어준다.
  a['Genre']=df1['Genre'] # 2번 for문에서 장르별로 구분하기 위해 Genre를 붙여준다.
  globals()['V_{}'.format(v)]= a # a에서 차례대로 받은 데이터를 t 리스트 순서대로 V_{t} 로 받아낸다.

  for i in range(len(genre)):  # 장르의 리스트 길이만큼 반복해준다. 0~ 11. 12 번 반복-----------------------------------------------------------------2번 for문
    condition=globals()['V_{}'.format(v)]['Genre']==genre[i] # 1번 for문에서 추출한 V_t에 장르의 유니크값을 추출한 genre 리스트(12개)로 조건문을 만들어준다.
    globals()['V{}'.format(i)] = globals()['V_{}'.format(v)][condition] # 1번 for문에서 나오는 V_{t} 값에 condition 조건을 걸고 2번 for문의 V{i}에 넣어준다.
    globals()['V{}'.format(i)].set_index('Genre',drop=True,inplace=True) # 위에서 내려온 V{i}데이터는 columns으로 Genre와 {t}데이터를 가지고 있다. 이대로 아노바를 하면 str 문제가 생기기 때문에 'Genre'컬럼 데이터를 인덱스로 바꿔준다.
    globals()['V_{}_{}'.format(v,i)]=globals()['V{}'.format(i)]
# 데이터 종류 : NA : V_NA_0~11
            #   EU : V_EU_0~11
            #   JP : V_JP_0~11
            # Other: V_Other_0~11

          

### 2.사전 검정

- 정규성 검정

각 지역별로 violinplot plot


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

data=[]
for j,v in enumerate(t):
  for i in range(len(genre)):
    data.append(globals()['V_{}_{}'.format(v,i)])


plt.figure(figsize=(3,3))
sns.violinplot(data=data[0:12],color='red')
plt.figure(figsize=(3,3))
sns.violinplot(data=data[13:25],color='red')
plt.figure(figsize=(3,3))
sns.violinplot(data=data[25:36],color='red')
plt.figure(figsize=(3,3))
sns.violinplot(data=data[36:48],color='red')

In [ ]:
# 샤피로 검정
# 귀무가설 : 이 집합들은 정규성을 가지고 있다.
from scipy import stats

t= ['NA','EU','JP','Other']

for j,v in enumerate(t):
  for i in range(len(genre)):
    shapiro_test = stats.shapiro(globals()['V_{}_{}'.format(v,i)]) # 샤피로 검정을 한다. 총 집합 개수인 4*12=48 개의 검정 결과이다.

    print("p-value=",shapiro_test[1])


- 검정 결과

일단 바이올린 플롯으로 봤을때도 심상치 않았는데 샤피로 검정을 통해 p-value가 0에 가깝게 나와 귀무가설인 '정규분포를 만족한다'를 기각하게 되었다. 따라서 아노바 검정은 사용이 불가능하다. 여기서 현재 내가 알고 있는 두 가지 방법이 있다.
1. 이상치 제거를 통한 정규 분포화 재검사
2. 비모수 검정


이상치는 위에서 언급했 듯 이상치가 특정 집단에 몰려 있어 제거하면 안 되는 값이라고 판단을 했다. 그리고 비모수 검정은 Kruskal-Wallis H 검정을 이용한다. 귀무가설인 '지역별로 각 장르는 차이가 없다'를 검정한다.

- Kruskal-Wallis H 검정
조건 
1. 비모수 검정
2. 세 집단 이상의 집단 분포를 비교
3. 모수 통계의 one way ANOVA 대신 사용한다.

In [ ]:
from scipy import stats

a=stats.kruskal(V_NA_0,V_NA_1,V_NA_2,V_NA_3,V_NA_4,V_NA_5,V_NA_6,V_NA_7,V_NA_8,V_NA_9,V_NA_10,V_NA_11)

b=stats.kruskal(V_EU_0,V_EU_1,V_EU_2,V_EU_3,V_EU_4,V_EU_5,V_EU_6,V_EU_7,V_EU_8,V_EU_9,V_EU_10,V_EU_11)

c=stats.kruskal(V_JP_0,V_JP_1,V_JP_2,V_JP_3,V_JP_4,V_JP_5,V_JP_6,V_JP_7,V_JP_8,V_JP_9,V_JP_10,V_JP_11)

d=stats.kruskal(V_Other_0,V_Other_1,V_Other_2,V_Other_3,V_Other_4,V_Other_5,V_Other_6,V_Other_7,V_Other_8,V_Other_9,V_Other_10,V_Other_11)

print("NA=",a)
print("EU=",b)
print("JP=",c)
print("Other=",d)



- 결론: 각 지역당 p-value 값은 0에 가까우며 귀무가설인 '각 지역별로 각 장르는 차이가 없다'를 기각하게 되었다. 사후검정으로 각 장르별 차이를 찾아내는 것은 목적을 이루는데 크게 의미가 없다. 그렇기에 누적값을 통해 지역별 선호하는 게임 장르를 확인하겠다.

## 3.누적값을 통해 지역별 선호하는 게임 장르 확인

In [ ]:
sum_list=[]
for i in range(len(genre)):  # 장르의 리스트 길이만큼 반복해준다.
  condition=df1['Genre']==genre[i] # 추출한 장르의 리스트를 이용하여 장르가 같은 데이터를 추출해주는 조건을 만든다. 
  a= df1[condition].sum() # 조건에 맞는 데이터를 추출하여 평균을 내어 a에 넣는다.
  sum_list.append(a) # 일정 횟수만큼 반복하면서 위에서 만든 비어있는 mean_list에 a 데이터를 쌓아준다.

df1_sum=pd.DataFrame(sum_list,index=genre) # 쌓인 데이터를 데이터 프레임으로 바꾸고 index를 추출한 장르의 리스트로 넣어준다.


df1_sum.drop('Genre',axis=1,inplace=True)

df1_sum

In [ ]:
sum_NA=df1_sum['NA_Sales']
sum_EU=df1_sum['EU_Sales']
sum_JP=df1_sum['JP_Sales']
sum_Other=df1_sum['Other_Sales']

import pandas as pd
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
sum_NA.plot.bar(x=sum_NA.index,color='blue')
plt.title('NA',fontsize=15)
plt.xticks(rotation=20)

plt.figure(figsize=(10,10))
sum_EU.plot.bar(x=sum_EU.index,color='green')
plt.title('EU',fontsize=15)
plt.xticks(rotation=20)

plt.figure(figsize=(10,10))
sum_JP.plot.bar(x=sum_JP.index,color='red')
plt.title('JP',fontsize=15)
plt.xticks(rotation=20)

plt.figure(figsize=(10,10))
sum_Other.plot.bar(x=sum_Other.index,color='yellow')
plt.title('Other',fontsize=15)
plt.xticks(rotation=20)

plt.show()


## 4.결론


NA : Action-sports-shooter-platform- misc<br><br>
EU: Action-sports-shooter-raching-misc<br><br>
JP : Role-playing-Action-sports-platform-Misc<br><br>
Other : Action-Sport-Shooter-Sports-Racing<br><br>

각 지역별로 위와 같은 순서로 누적 합계가 뚜렷한 걸로 보아 지역별 선호도가 있다고 볼 수 있다.

# 연도별 게임의 트렌드

## 1.연도별 장르 빈도수 중 최댓값

In [ ]:
Y_G_C=df.groupby(['Year','Genre']).size().reset_index(name='Count')

Y_G_C_max = Y_G_C.groupby(['Year'])['Count'].transform(max) # 찾아보기 transform(max)

condition = Y_G_C['Count']==Y_G_C_max

Y_G_C_max1= Y_G_C[condition]

Y_G_C_max1

- 연도별 최대 장르의 빈도수의 최댓값 그래프

In [ ]:
import seaborn as sns

plt.figure(figsize=(25,5))
sns.barplot(x = Y_G_C_max1['Year'], y = Y_G_C_max1['Count'],hue=Y_G_C_max1['Genre'])

연도별 최대 장르의 빈도수를 그래프로 그렸고1994년부터 2002년까지 Sports, 2003년부터 2015년까지 Action 장르가 높은 빈도로 나타났다.


## 2.연도별 장르 출고량 중 최댓값

In [ ]:

Total_Sales= df['NA_Sales']+df['EU_Sales']+df['JP_Sales']+df['Other_Sales']
Year=df['Year']
Gen=df['Genre']

Year=pd.DataFrame(Year)
Total_Sales=pd.DataFrame(Total_Sales)

Year['Total_Sales']=Total_Sales
Year['Genre']=Gen

Y_S_G_max= Year.groupby('Year')['Total_Sales'].transform(max)

condition = Year['Total_Sales']==Y_S_G_max
Y_S_G_max1=Year[condition]

Y_S_G_max1.sort_values('Year')

- 연도별 최대 출고 장르

In [ ]:
import seaborn as sns

plt.figure(figsize=(25,5))
g=sns.barplot(x = Y_S_G_max1['Year'], y = Y_S_G_max1['Total_Sales'],hue=Y_S_G_max1['Genre'])

연도별 최대 출고를 달성한 장르를 그래프로 나타냈다. 특징으로는 2005년에 가장 높은 출고값으로 Sports 장르를 기록하였고 그 이후로는 Racing, Sprots,Misc, Action , Shooter 순으로 연도별 최대 출고량을 기록하였다.

1987년 부터 꾸준히 Sports 게임을 발매했고, 2003년부터는 Action 장르가 가장 많이 발매되었다. 하지만 출고량을 따지면 2005년에 Sprots 장르가 대박이났고, Racing , Sports, Shooter ,Action ,Misc 등 다양하게 최대 출고량을 차지하였다. <br>
결론: Sports와 Action 가 최근 약20년동안 트랜드였지만, 5개년도를 제외하고는 다른 장르에 최대 출고량을 빼았겼다.

# 출고량이 높은 게임에 대한 분석

## 1.최대 출고량 TOP10

In [ ]:
df3= df[['Year','Genre','Platform','Publisher']]

df3['Total_Sales']=Total_Sales

Top10=df3.sort_values('Total_Sales',ascending=False).head(10)

Top10.sort_values('Year')

- 탑 10 최대 출고 장르

In [ ]:
import seaborn as sns

plt.figure(figsize=(15,5))
g=sns.barplot(x = Top10['Year'], y = Top10['Total_Sales'],hue=Top10['Genre'])

전체 판매량으로 연도별 장르 탑 10을 확인 했을 때, 2005년 Sport가 독보적으로 높은 판매량을 가지고 있고, 그 이후로는 Platform, Racing ,Role-playing,Puzzle,Misc,Shooter이 각각의 연도에서 비슷한 출고량을 가지고 있다. 

- 탑 10 최대 출고 플랫폼

In [ ]:
import seaborn as sns

plt.figure(figsize=(15,5))
g=sns.barplot(x = Top10['Year'], y = Top10['Total_Sales'],hue=Top10['Platform'])

전체 판매량으로 연도별 플랫폼 탑10을 확인 했을 때, 전체적으로 1984년부터 2008년까지 NES-GB-Will-DS-Will 순으로 최대 판매량을 가지고 있다. 그리고 모두 배급사는 Nintendo 이다.

탑 10에서는 플랫폼이 전부 Nintendo에 몰려 있으므로 탑 30을 확인해보겠다.

## 2.최대 출고량 TOP30

- 탑 30 최대 출고 장르

In [ ]:
Top30=df3.sort_values('Total_Sales',ascending=False).head(30)

plt.figure(figsize=(15,5))
g=sns.barplot(x = Top30['Year'], y = Top30['Total_Sales'],hue=Top30['Genre'])

- 탑 30 최대 출고 플랫폼

In [ ]:
plt.figure(figsize=(15,5))
g=sns.barplot(x = Top30['Year'], y = Top30['Total_Sales'],hue=Top30['Platform'])

탑 30 최대 출고 배급사를 확인해보았다.

In [ ]:
plt.figure(figsize=(15,5))
g=sns.barplot(x = Top30['Year'], y = Top30['Total_Sales'],hue=Top30['Publisher'])

탑 30을 봤을 때, 최대 출고 장르는 다양한 장르들이 상위권을 가지고 있어 큰 의미가 없다고 판단된다. 하지만 탑30 최대 출고 플랫폼을 봤을 때 2010년 이전까지 NinTendo사 will,DS 등의 플랫폼의 기록이 두드러지지만 그 이후에는 플레이스테이션이나 xbox의 플랫폼의 판매량이 두드러진다.

## 3.최근 10년간 최대 출고량 TOP10

In [ ]:
condition= ((df3['Year']<=2020) & (df3['Year']>2010))

year10=df3[condition].sort_values('Total_Sales',ascending=False)

Top10_y= year10.head(10)

Top10_y

- 최근 10년간 탑 10 최대 출고 장르

In [ ]:
import seaborn as sns

plt.figure(figsize=(15,5))
g=sns.barplot(x = Top10_y['Year'], y = Top10_y['Total_Sales'],hue=Top10_y['Genre'])

최근 10년간 연도별 최대 출고량을 장르 기준에서 보았다. 2012년부터 2014년까지 Action이 최대 판매량을 가진 후 슈터는 전 후에서 높은 값을, 그리고 롤플레잉, 레이싱이 그 뒤를 잇고 있다.

- 최근 10년간 탑 10 최대 출고 플랫폼

In [ ]:
import seaborn as sns

plt.figure(figsize=(15,5))
g=sns.barplot(x = Top10_y['Year'], y = Top10_y['Total_Sales'],hue=Top10_y['Platform'])

최근 10년간 연도별 최대 출고량을 플랫폼 기준에서 보았다. 매년 플레이스테이션은 최대 출고량을 가지고 있고 2014년까지 xbox는 점점 플레이스테이션에 밀리는 경향을 보여주고 2014년에 3DS가 다시 높은 출고량을 보여준다.

## 4.최근 10년간 최대 출고량 TOP30

- 최근 10년간 탑 30 최대 출고 장르

In [ ]:
Top30_y=year10.head(30)

plt.figure(figsize=(15,5))
g=sns.barplot(x = Top30_y['Year'], y = Top30_y['Total_Sales'],hue=Top30_y['Genre'])


탑 30으로 보았을 때, 매년 상위권을 유지하는 장르로 Shooter이 눈에 띈다.

- 최근 10년간 탑 30 최대 출고 플랫폼

In [ ]:
plt.figure(figsize=(15,5))
g=sns.barplot(x = Top30_y['Year'], y = Top30_y['Total_Sales'],hue=Top30_y['Platform'])

플레이스테이션은 매년 가장 높은 수치를 보여주고 있다.

## 5.최근 10년 각 지역 별  장르 TOP 30

- NA

In [ ]:
df_NA= df[['Year','Genre','Platform','NA_Sales']]

df_NA

condition= ((df_NA['Year']<=2020) & (df_NA['Year']>2010))

df_NA_10=df_NA[condition].sort_values('NA_Sales',ascending=False)

a=df_NA_10.head(30)

plt.figure(figsize=(15,5))
g=sns.barplot(x = a['Year'], y = a['NA_Sales'],hue=a['Genre'])

- EU

In [ ]:
df_EU= df[['Year','Genre','Platform','EU_Sales']]

df_EU

condition= ((df_EU['Year']<=2020) & (df_EU['Year']>2010))

df_EU_10=df_EU[condition].sort_values('EU_Sales',ascending=False)

b=df_EU_10.head(30)

plt.figure(figsize=(15,5))
g=sns.barplot(x = b['Year'], y = b['EU_Sales'],hue=b['Genre'])

- JP

In [ ]:
df_JP= df[['Year','Genre','Platform','JP_Sales']]

df_JP

condition= ((df_JP['Year']<=2020) & (df_JP['Year']>2010))

df_JP_10=df_JP[condition].sort_values('JP_Sales',ascending=False)

c=df_JP_10.head(30)

plt.figure(figsize=(15,5))
g=sns.barplot(x = c['Year'], y = c['JP_Sales'],hue=c['Genre'])

- Other

In [ ]:
df_Other= df[['Year','Genre','Platform','Other_Sales']]

df_Other

condition= ((df_Other['Year']<=2020) & (df_Other['Year']>2010))

df_Other_10=df_Other[condition].sort_values('Other_Sales',ascending=False)

d=df_Other_10.head(30)

plt.figure(figsize=(15,5))
g=sns.barplot(x = d['Year'], y = d['Other_Sales'],hue=d['Genre'])

# 장르별 플랫폼의 연관성

귀무가설 = 장르별 플랫폼의 연관성이 없다.

In [ ]:
import scipy.stats as stats

Total_Sales= df['NA_Sales']+df['EU_Sales']+df['JP_Sales']+df['Other_Sales']

df4= df[['Genre','Platform']]

df4['Total_Sales']=Total_Sales




stats.chi2_contingency(pd.crosstab(index=df4['Genre'],columns=df4['Platform']))

# p-value = 0.0 -> 장르별 플랫폼의 연관성이 있다.

## 장르별 플랫폼의 출고량 확인


In [ ]:
import seaborn as sns

t=['Action','Shooter','Sports','Role-Playing','Racing','Platform','Misc']

for i ,j in enumerate(t):
   globals()['condition{}'.format(i)] =df4['Genre']==j


a=pd.crosstab(index=df4['Genre'],columns=df4['Platform'])

ax = sns.heatmap(a)
sns.set(rc = {'figure.figsize':(20,10)})

plt.title('Genre and Platform', fontsize=10)

plt.show()






# 종합

지역별 선호<br>
JP 지역을 제외하고는 Action Sports Shooter 순서로 선호도가 강하다. 특별히 JP에서는 롤플레잉이 제일 높은 선호도를 가지고 있고 Action과 Sports가 그 뒤를 잇는다.


트랜드<br>
2002년 전까지는 sports, 그 이후에는 Action이 최대 출고량을 가지고 있는 것으로 보아 트랜드라고 볼 수 있을 것 같다. 하지만 최대 출고량을 확인했을 때는 2002년 이후 2015년까지 4개년을 제외하곤 Action이 최대 판매량을 달성하지 못했다. 최근 10년인 2010년부터 탑 30을 확인해보면 다양한 장르가 팔리지만 그 중 Shooter 장르가 매년 상위권을 유지하고 있다.


최대 출고량을 기록한 게임<br>
탑 30을 봤을 때, 모든 장르들이 다양하게 최대 출고량을 달성했다. 그리고 최대 플랫폼과 배급사를 확인 해본 결과 2009년까지 닌텐도사의 독주가 눈에 띄지만 2010년부터는 다른 배급사에 밀리고 있다.

플랫폼<br>
최근 10년 기록을 봤을 때, 매년 플레이스테이션의 독주가 눈에 띈다.

최근 10년간 지역별 선호도 TOP 30<br>

NA - Action이 우세<br>
EU - Sports가 우세  <br>
JP - 14년까지는 Role-playing이 우세했지만 이후로는 Action 이 우세<br>
Other- Shooter와 Sports가 우세

특징으로는 NA와 EU가 출고량 기준이 JP 와 Other 보다 높다. 

# 결론

- 플레이스테이션 플랫폼을 가진 Shooter 게임을 설계한다.

장르는 최근 10년을 봤을 때 Sports보다는 Action과 Shooter의 대세가 눈에 띈다. Sports는 EU 지역에서만 뚜렷하게 팔리고 있기 때문에 전체를 아우르기는 힘들 것 같다. 그 중에서 Action은 Shooter와 비슷한 판매량을 보이고 있지만, Action은 게임 개발 빈도수가 제일 높은 장르 중 하나기 때문에 출시 후에는 많은 게임들과 경쟁을 해야 할 것이다. 따라서 Shooter 장르를 선택하겠다.

플랫폼은 2010년 이 후부터는 플레이스테이션 게임의 폭발적인 출고량이 지속적으로 나타나고 있다. 플레이스테이션 게임이 최대 판매량에 주로 있다는 것은 많은 사람들이 최신 플레이스테이션 기기를 가지고 있다는 뜻이며 게임 출시를 했을 때 많은 사람들이 접할 수 있다는 뜻이다. 또한 히트맵 그래프를 통해 보면 Shooter 장르와 플레이스테이션의 궁합이 나쁘지 않다.